<p><b>Predição preço de ações utilizando LSTM.</b></p>

#### Redes Neurais

https://pathmind.com/wiki/neural-network


 #### LSTM

https://pathmind.com/wiki/lstm#recurrent

#### Construção da LSTM

In [1]:
#Bibliotecas para manipulação dos dados
import pandas as pd
import numpy as np
from datetime import datetime

#Bibliotecas de gráfico
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
plt.style.use('ggplot')

#Bibliotecas para construímos a LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, LeakyReLU
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from functions import get_finance_data

#Biblioteca para retirar avisos
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'keras'

In [4]:
import tensorflow

#### Tratamento de Dados

In [ ]:
#Leitura dos dados
dados=pd.read_csv("Apple_stocks.csv",sep=",")

In [ ]:
#Conhecendo os dados que iremos trabalhar
dados.head()

In [ ]:
dados.dtypes

<p> Podemos observar que nossos dados não estão em um formato que podem ser utilizados em uma LSTM, pois por padrão as sequências de dados de <i>input</i> devem ser numéricos.</p>
<p>Além disso, a coluna <i>Date</i> está com um formato diferente do desejável para construírmos nossa Série Temporal.<p>


In [ ]:
#Vamos criar uma cópia dos dados originais para podermos manipular os dados.
df = dados.copy()

In [ ]:
#Renomeando as colunas
df.columns = ["Date","Close","Volume","Open","High","Low"]

O objetivo da nossa LSTM vai ser prever o preço de fechamento da ação, com isso precisamos transformar os dados da coluna <i><b>Close</b></i> de <i>object</i> para <i>float</i>.

In [ ]:
#Definindo a função para alteramos as datas
def to_string_date(x:str):
    return pd.to_datetime(
                datetime.strftime(
                      datetime.strptime(x.replace("/","-"),"%m-%d-%Y"),"%Y-%m-%d"))

In [ ]:
#Definindo a função para alteramos o valor de fechamento
def to_float(x:str):
    return float(x[x.find("$")+1:])

In [ ]:
#Aplicamos as funções nas colunas que precisam de transformação
df["Date"] = df["Date"].apply(to_string_date)
df["Close"] = df["Close"].apply(to_float)
df = df.sort_values('Date')

In [ ]:
#Visualização dos dados após alteração
df.head()

<p>Agora que fizemos o tratamento das colunas que iremos utilizar, precisamos colocar nossos dados no formato de uma Série Temporal.</p>


In [ ]:
df.index = df["Date"]
df = df["Close"]

<p>Após criada nossa Série, vamos fazer um plot dos dados.</p>

In [ ]:
#Vamos analisar o comportamento da nossa variável resposta
plt.figure(figsize=(16,8))
plt.ylabel("Preço de Fechamento em USD($)",fontsize=15)
plt.title('Série de Preço de Fechamento',fontsize=25)
plt.xticks(rotation= 45,fontsize=15)
plt.plot(df)

<p>Ao analisarmos o gráfico podemos perceber uma tedência na nossa série, além de uma grande discrepância entre os valores observados. </p>
<p>Para solucionarmos o problema da diferença entre os valores, vamos usar uma técnica de normalização.</p>

In [ ]:
dataset = df.values.reshape((-1,1))
scaler = MinMaxScaler(feature_range=(0, 1)) 
dataset = scaler.fit_transform(dataset)

<p>Antes de iniciarmos o processo de modelagem propriamente dito, temos que decidir quantas observações do passado vamos utilizar para estimarmos o valor futuro.</p>
<p>Além disso, temos que decidir quantos passos à frente vamos querer estimar.</p>

In [ ]:
'''Para treinar nossa rede, usaremos um intervalo de tempo para prevermos o nosso próximo valor'''
look_back = 5
future_target = 1

<p>Então, seguindo o processo de modelagem, vamos separar nossos dados em dados de treino e teste. Nesse caso, vamos usar 70% dos dados como dados de teste.</p>

In [ ]:
'''Separando os dados em treino e teste.'''
tam = int(len(dataset) * 0.70)
dataset_teste = dataset[tam:]
dataset_treino = dataset[:tam]

<p>Por padrão, Redes Neurais necessitam que os dados de <i>input</i> estejam na forma matricial, de preferência uma matriz tridimensional.<p>

In [ ]:
#Função de transformação dos dados
def process_data(data, look_back, forward_days,jump=1):
    X,Y = [],[]
    for i in range(0,len(data) -look_back -forward_days +1, jump):
        X.append(data[i:(i+look_back)])
        Y.append(data[(i+look_back):(i+look_back+forward_days)])
    return np.array(X),np.array(Y)


In [ ]:
X, y = process_data(dataset_treino,look_back,future_target)
y = np.array([list(a.ravel()) for a in y])

x_test, y_test = process_data(dataset_teste,look_back,future_target)
y_test = np.array([list(a.ravel()) for a in y_test])

<p>Realizamos esse processo para permitir que a rede possa identificar que cada conjunto de valores de X_train gere o valor de y_train, ou seja, o input x_train[0] gera o output y_train[0] e assim sussecivamente.<p>

In [ ]:
X[0]

In [ ]:
y[0]

<p>Agora, vamos separar nosso conjunto de dados de teste e separar um porcentagem para validação.</p>

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.20, random_state=42)

<p>No próximo passo vamos contruir a rede neural propriamente dita.</p>
<p>Iremos usar alguns paramêtros padrões, pois o processo de tunning dos parâmetros é um processo que exige testes e investigações mais profundas sobre os dados.</p>

In [ ]:
#Definindo os números de neurônios por camada
n_first = 128
EPOCHS = 50
#Construido o modelo
model = Sequential()
model.add(LSTM(n_first,input_shape = (look_back,1)))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(X_train,y_train,epochs=EPOCHS,validation_data=(X_validate,y_validate),shuffle=False,batch_size=2, verbose=2)

<p>Após o treinamento do nosso modelo, podemos verificar o comportamento da função custo nos dados de validação e nos dados de teste.</p>
<p>A análise do comportamento das curvas podem ser usados como indicador de <i>underfitting</i> ou <i>overfitting</i>.</p>

In [ ]:
#Gráfico do resultado da função perda por epochs
plt.figure(figsize = (15,10))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylabel("Número de Epochs")
plt.legend(loc='best',fontsize=15)
plt.show()

<p>Em seguida, usamos nosso modelo escolhido para seguirmos para o processo de predição.</p>

In [ ]:
#Salvando os valores preditos
Xt = model.predict(x_test)

<p>Por fim, podemos identificar o comportamento do nosso modelo na série.</p>
<p>Podemos observar que nosso modelo se comportou bem diante das configurações que escolhemos.</p>

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(scaler.inverse_transform(y_test.reshape(-1,1)),c='b', label='Teste')
plt.plot(scaler.inverse_transform(Xt.reshape(-1,1)), c='r',label='Predito')
plt.ylabel("Preço de Fechamento em USD($)")
plt.legend(loc='best')
plt.show()

#### Próximos passos

<p>Vimos que treinar uma Rede Neural não é uma tarefa de outro mundo, contudo exige conhecimento e prática para aperfeiçoar a rede.</p>
